In [85]:
import pandas as pd
import numpy as np
import xgboost as xgb
import statistics
from statistics import mean
from sklearn.model_selection import GroupKFold
from sklearn import tree
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [86]:
# import csv

X = pd.read_csv("/Users/sky/Desktop/ca1-dataset.csv")

In [87]:
X

Unique-id       namea OffTask  \
0    awagner-closeloop-ins_h1zaz4-03.30.2011_at_13:...  stuZgy46jl       N   
1    awagner-closeloop-ins_h1zaz4-03.30.2011_at_13:...  stuZ77be0l       N   
2    awagner-closeloop-ins_h1zaz4-03.30.2011_at_13:...  stuZ5lp7k7       N   
3    awagner-closeloop-ins_h1zaz4-03.30.2011_at_13:...  stuZ9idg5l       N   
4    awagner-closeloop-ins_h1zaz4-03.30.2011_at_13:...  stuZ4lil98       N   
..                                                 ...         ...     ...   
758  jkalka-cog_tutor_ryans_study-ins_iellyr-04.04....  stuZ7rnhoz       N   
759  jkalka-cog_tutor_ryans_study-ins_iellyr-04.04....  stuZc0q0x3       N   
760  jkalka-cog_tutor_ryans_study-ins_iellyr-04.04....  stuZq83gfs       N   
761  jkalka-cog_tutor_ryans_study-ins_iellyr-04.04....  stuZetcek4       Y   
762  jkalka-cog_tutor_ryans_study-ins_iellyr-04.04....  stuZc0q0x3       N   

     Avgright  Avgbug  Avghelp  Avgchoice  Avgstring  Avgnumber  Avgpoint  \
0    1.000000     0.0        0          0          0          0         0   
1    1.000000     0.0        0          0          0          0         0   
2    1.000000     0.0        0          0          0          0         0   
3    0.833333     0.0        0          0          0          0         0   
4    1.000000     0.0        0          0          0          0         0   
..        ...     ...      ...        ...        ...        ...       ...   
758  0.000000     0.0        0          0          0          0         0   
759  1.000000     0.0        0          0          0          0         0   
760  1.000000     0.0        0          0          0          0         0   
761  1.000000     0.0        0          0          0          0         0   
762  0.666667     0.0        0          0          0          0         0   

     ...  Avghelppct-up  Avgwrongpct-up  Avgtimeperact-up  AvgPrev3Count-up  \
0    ...              0        0.000000         12.000000          0.000000   
1    ...              0        0.000000          8.750000          0.000000   
2    ...              0        0.000000         26.083333          0.000000   
3    ...              0        0.166667          4.902778          0.000000   
4    ...              0        0.000000          5.875000          0.000000   
..   ...            ...             ...               ...               ...   
758  ...              0        4.000000         17.500000          2.000000   
759  ...              0        0.000000          6.533333          0.000000   
760  ...              0        0.000000         17.076923          0.000000   
761  ...              0        0.000000         16.777778          0.000000   
762  ...              0        0.333333          8.148148          0.333333   

     AvgPrev5Count-up  Avgrecent8help  Avg recent5wrong  Avgmanywrong-up  \
0            0.000000               0          1.000000              0.0   
1            0.000000               0          0.000000              0.0   
2            0.000000               0          1.000000              0.0   
3            0.000000               0          0.000000              0.0   
4            0.000000               0          0.000000              0.0   
..                ...             ...               ...              ...   
758          4.000000               0          0.000000              0.0   
759          0.000000               0          0.000000              0.0   
760          0.000000               0          2.000000              0.0   
761          0.000000               0          0.000000              0.0   
762          0.333333               0          1.666667              0.0   

     AvgasymptoteA-up  AvgasymptoteB-up  
0                   0                 0  
1                   0                 0  
2                   0                 0  
3                   0                 0  
4                   0                 0  
..                ...               ...  
758                 0      

In [88]:
# replace the "Y" and "N" to number 1 and 0 in column "OffTask"

K = X["OffTask"].replace(["Y", "N"], [1, 0])

In [89]:
# remove id identifier data "Unique-id" as well as "namea", and the label "off task"

X1 = X.drop(columns=["Unique-id", "namea", "OffTask"], axis=1)

In [90]:
X1

Avgright  Avgbug  Avghelp  Avgchoice  Avgstring  Avgnumber  Avgpoint  \
0    1.000000     0.0        0          0          0          0         0   
1    1.000000     0.0        0          0          0          0         0   
2    1.000000     0.0        0          0          0          0         0   
3    0.833333     0.0        0          0          0          0         0   
4    1.000000     0.0        0          0          0          0         0   
..        ...     ...      ...        ...        ...        ...       ...   
758  0.000000     0.0        0          0          0          0         0   
759  1.000000     0.0        0          0          0          0         0   
760  1.000000     0.0        0          0          0          0         0   
761  1.000000     0.0        0          0          0          0         0   
762  0.666667     0.0        0          0          0          0         0   

     Avgpchange    Avgtime  AvgtimeSDnormed  ...  Avghelppct-up  \
0      0.000000  12.000000         0.446098  ...              0   
1      0.000000   7.500000        -0.260956  ...              0   
2      0.000000  25.333333         0.301574  ...              0   
3      0.000000   3.500000        -0.554844  ...              0   
4      0.000000   5.250000        -0.525770  ...              0   
..          ...        ...              ...  ...            ...   
758    1.000000  56.000000         5.110030  ...              0   
759    0.000000   6.000000         0.042553  ...              0   
760    0.000000  14.000000         0.339286  ...              0   
761    0.000000   5.000000        -0.329097  ...              0   
762    0.333333  11.000000         0.813424  ...              0   

     Avgwrongpct-up  Avgtimeperact-up  AvgPrev3Count-up  AvgPrev5Count-up  \
0          0.000000         12.000000          0.000000          0.000000   
1          0.000000          8.750000          0.000000          0.000000   
2          0.000000         26.083333          0.000000          0.000000   
3          0.166667          4.902778          0.000000          0.000000   
4          0.000000          5.875000          0.000000          0.000000   
..              ...               ...               ...               ...   
758        4.000000         17.500000          2.000000          4.000000   
759        0.000000          6.533333          0.000000          0.000000   
760        0.000000         17.076923          0.000000          0.000000   
761        0.000000         16.777778          0.000000          0.000000   
762        0.333333          8.148148          0.333333          0.333333   

     Avgrecent8help  Avg recent5wrong  Avgmanywrong-up  AvgasymptoteA-up  \
0                 0          1.000000              0.0                 0   
1                 0          0.000000              0.0                 0   
2                 0          1.000000              0.0                 0   
3                 0          0.000000              0.0                 0   
4                 0          0.000000              0.0                 0   
..              ...               ...              ...               ...   
758               0          0.000000              0.0                 0   
759               0          0.000000              0.0                 0   
760               0          2.000000              0.0                 0   
761               0          0.000000              0.0                 0   
762               0          1.666667              0.0                 0   

     AvgasymptoteB-up  
0                   0  
1                   0  
2                   0  
3                   0  
4                   0  
..                ...  
758                 0  
759                 0  
760                 0  
761                 0  
762                 0  

[763 rows x 24 columns]

In [91]:
# demonsrate the detailed information of the dataset

X1.describe()

Avgright      Avgbug  Avghelp  Avgchoice  Avgstring  Avgnumber  \
count  763.000000  763.000000    763.0      763.0      763.0      763.0   
mean     0.713072    0.055168      0.0        0.0        0.0        0.0   
std      0.394293    0.184326      0.0        0.0        0.0        0.0   
min      0.000000    0.000000      0.0        0.0        0.0        0.0   
25%      0.500000    0.000000      0.0        0.0        0.0        0.0   
50%      1.000000    0.000000      0.0        0.0        0.0        0.0   
75%      1.000000    0.000000      0.0        0.0        0.0        0.0   
max      1.000000    1.000000      0.0        0.0        0.0        0.0   

       Avgpoint  Avgpchange     Avgtime  AvgtimeSDnormed  ...  Avghelppct-up  \
count     763.0  763.000000  763.000000       763.000000  ...          763.0   
mean        0.0    0.283065   14.117770         0.224450  ...            0.0   
std         0.0    0.386746   15.623914         1.317254  ...            0.0   
min         0.0    0.000000   -1.000000        -1.000000  ...            0.0   
25%         0.0    0.000000    6.250000        -0.368185  ...            0.0   
50%         0.0    0.000000    9.666667        -0.121190  ...            0.0   
75%         0.0    0.500000   16.000000         0.306074  ...            0.0   
max         0.0    1.000000  205.000000        13.541537  ...            0.0   

       Avgwrongpct-up  Avgtimeperact-up  AvgPrev3Count-up  AvgPrev5Count-up  \
count      763.000000        763.000000        763.000000        763.000000   
mean         1.483525         14.241320          0.446376          0.574707   
std          4.361797         10.366487          0.796067          1.097031   
min          0.000000          0.000000          0.000000          0.000000   
25%          0.000000          7.838235          0.000000          0.000000   
50%          0.000000         11.000000          0.000000          0.000000   
75%          0.750000         17.080128          0.500000          0.500000   
max         47.666667        114.333333          3.000000          5.000000   

       Avgrecent8help  Avg recent5wrong  Avgmanywrong-up  AvgasymptoteA-up  \
count           763.0        763.000000       763.000000             763.0   
mean              0.0          0.914248         0.044288               0.0   
std               0.0          1.038397         0.179562               0.0   
min               0.0          0.000000         0.000000               0.0   
25%               0.0          0.000000         0.000000               0.0   
50%               0.0          0.750000         0.000000               0.0   
75%               0.0          1.500000         0.000000               0.0   
max               0.0          5.000000         1.000000               0.0   

       AvgasymptoteB-up  
count             763.0  
mean                0.0  
std                 0.0  
min                 0.0  
25%                 0.0  
50%                 0.0  
75%                 0.0  
max                 0.0  

[8 rows x 24 columns]

In [92]:
# drop the data "Avghelp", "Avgchoice","Avgstring","Avgnumber","Avgpoint","Avghelppct-up","Avgrecent8help","AvgasymptoteA-up","AvgasymptoteB-up"
# since the value of all these variables are 0


X2 = X1.drop(columns=["Avghelp", "Avgchoice","Avgstring","Avgnumber","Avgpoint","Avghelppct-up","Avgrecent8help","AvgasymptoteA-up","AvgasymptoteB-up"], axis=1)

In [93]:
X2

Avgright  Avgbug  Avgpchange    Avgtime  AvgtimeSDnormed  \
0    1.000000     0.0    0.000000  12.000000         0.446098   
1    1.000000     0.0    0.000000   7.500000        -0.260956   
2    1.000000     0.0    0.000000  25.333333         0.301574   
3    0.833333     0.0    0.000000   3.500000        -0.554844   
4    1.000000     0.0    0.000000   5.250000        -0.525770   
..        ...     ...         ...        ...              ...   
758  0.000000     0.0    1.000000  56.000000         5.110030   
759  1.000000     0.0    0.000000   6.000000         0.042553   
760  1.000000     0.0    0.000000  14.000000         0.339286   
761  1.000000     0.0    0.000000   5.000000        -0.329097   
762  0.666667     0.0    0.333333  11.000000         0.813424   

     Avgtimelast3SDnormed  Avgtimelast5SDnormed  Avgnotright  \
0                0.288201              0.454262     0.000000   
1               -0.774506              0.000000     0.000000   
2               -1.529301             -0.508076     0.000000   
3               -1.366454             -2.018076     0.166667   
4               -1.829775             -2.621184     0.000000   
..                    ...                   ...          ...   
758             -0.979703             -0.514252     1.000000   
759              0.581949              1.955626     0.000000   
760              1.769959              1.728446     0.000000   
761              3.106351              4.530658     0.000000   
762              0.602398             -0.155730     0.333333   

     Avghowmanywrong-up  Avgwrongpct-up  Avgtimeperact-up  AvgPrev3Count-up  \
0              0.000000        0.000000         12.000000          0.000000   
1              0.000000        0.000000          8.750000          0.000000   
2              0.000000        0.000000         26.083333          0.000000   
3              0.166667        0.166667          4.902778          0.000000   
4              0.000000        0.000000          5.875000          0.000000   
..                  ...             ...               ...               ...   
758            8.000000        4.000000         17.500000          2.000000   
759            0.000000        0.000000          6.533333          0.000000   
760            0.000000        0.000000         17.076923          0.000000   
761           10.000000        0.000000         16.777778          0.000000   
762            1.000000        0.333333          8.148148          0.333333   

     AvgPrev5Count-up  Avg recent5wrong  Avgmanywrong-up  
0            0.000000          1.000000              0.0  
1            0.000000          0.000000              0.0  
2            0.000000          1.000000              0.0  
3            0.000000          0.000000              0.0  
4            0.000000          0.000000              0.0  
..                ...               ...              ...  
758          4.000000          0.000000              0.0  
759          0.000000          0.000000              0.0  
760          0.000000          2.000000              0.0  
761          0.000000          0.000000              0.0  
762          0.333333          1.666667              0.0  

[763 rows x 15 columns]

In [94]:
X2.corr()

Avgright    Avgbug  Avgpchange   Avgtime  \
Avgright              1.000000 -0.341996   -0.527984 -0.079358   
Avgbug               -0.341996  1.000000    0.022625  0.031367   
Avgpchange           -0.527984  0.022625    1.000000  0.065769   
Avgtime              -0.079358  0.031367    0.065769  1.000000   
AvgtimeSDnormed      -0.127433  0.056050    0.103155  0.874219   
Avgtimelast3SDnormed -0.066564 -0.033338    0.072013  0.247185   
Avgtimelast5SDnormed -0.078406 -0.014893    0.088300  0.254885   
Avgnotright          -1.000000  0.341996    0.527984  0.079358   
Avghowmanywrong-up   -0.485482  0.059668    0.439635  0.058543   
Avgwrongpct-up       -0.515952  0.067259    0.326488  0.070352   
Avgtimeperact-up     -0.019031  0.051913   -0.016287  0.436722   
AvgPrev3Count-up     -0.550912  0.009742    0.834952  0.014781   
AvgPrev5Count-up     -0.530121 -0.008021    0.785967  0.027488   
Avg recent5wrong     -0.441910  0.007995    0.553494  0.057893   
Avgmanywrong-up      -0.381036 -0.018362    0.420498  0.021850   

                      AvgtimeSDnormed  Avgtimelast3SDnormed  \
Avgright                    -0.127433             -0.066564   
Avgbug                       0.056050             -0.033338   
Avgpchange                   0.103155              0.072013   
Avgtime                      0.874219              0.247185   
AvgtimeSDnormed              1.000000              0.211772   
Avgtimelast3SDnormed         0.211772              1.000000   
Avgtimelast5SDnormed         0.223248              0.896690   
Avgnotright                  0.127433              0.066564   
Avghowmanywrong-up           0.091547             -0.023798   
Avgwrongpct-up               0.119623             -0.002001   
Avgtimeperact-up             0.281118              0.399567   
AvgPrev3Count-up             0.049272              0.041337   
AvgPrev5Count-up             0.076283             -0.001671   
Avg recent5wrong             0.062885              0.094552   
Avgmanywrong-up              0.071888             -0.026047   

                      Avgtimelast5SDnormed  Avgnotright  Avghowmanywrong-up  \
Avgright                         -0.078406    -1.000000           -0.485482   
Avgbug                           -0.014893     0.341996            0.059668   
Avgpchange                        0.088300     0.527984            0.439635   
Avgtime                           0.254885     0.079358            0.058543   
AvgtimeSDnormed                   0.223248     0.127433            0.091547   
Avgtimelast3SDnormed              0.896690     0.066564           -0.023798   
Avgtimelast5SDnormed              1.000000     0.078406            0.011462   
Avgnotright                       0.078406     1.000000            0.485482   
Avghowmanywrong-up                0.011462     0.485482            1.000000   
Avgwrongpct-up                    0.031546     0.515952            0.651608   
Avgtimeperact-up                  0.369893     0.019031           -0.046664   
AvgPrev3Count-up                  0.056941     0.550912            0.495893   
AvgPrev5Count-up                  0.022140     0.530121            0.557018   
Avg recent5wrong                  0.132207     0.441910            0.359736   
Avgmanywrong-up                  -0.009911     0.381036            0.502704   

                      Avgwrongpct-up  Avgtimeperact-up  AvgPrev3Count-up  \
Avgright                   -0.515952         -0.019031         -0.550912   
Avgbug                      0.067259          0.051913          0.009742   
Avgpchange                  0.326488         -0.016287          0.834952   
Avgtime                     0.070352          0.436722          0.014781   
AvgtimeSDnormed             0.119623          0.281118          0.049272   
Avgtimelast3SDnormed       -0.002001          0.399567          0.041337   
Avgtimelast5SDnormed        0.031546          0.369893          0.056941   
Avgnotright                 0.515952          0.019031          0.550912   
Avghowm

In [95]:
# drop the column "Avgnotright", since the correlation between the "Avgright" and "Avgnotright" is -1

X3 = X2.drop(columns=["Avgnotright"], axis=1)

In [96]:
X3

Avgright  Avgbug  Avgpchange    Avgtime  AvgtimeSDnormed  \
0    1.000000     0.0    0.000000  12.000000         0.446098   
1    1.000000     0.0    0.000000   7.500000        -0.260956   
2    1.000000     0.0    0.000000  25.333333         0.301574   
3    0.833333     0.0    0.000000   3.500000        -0.554844   
4    1.000000     0.0    0.000000   5.250000        -0.525770   
..        ...     ...         ...        ...              ...   
758  0.000000     0.0    1.000000  56.000000         5.110030   
759  1.000000     0.0    0.000000   6.000000         0.042553   
760  1.000000     0.0    0.000000  14.000000         0.339286   
761  1.000000     0.0    0.000000   5.000000        -0.329097   
762  0.666667     0.0    0.333333  11.000000         0.813424   

     Avgtimelast3SDnormed  Avgtimelast5SDnormed  Avghowmanywrong-up  \
0                0.288201              0.454262            0.000000   
1               -0.774506              0.000000            0.000000   
2               -1.529301             -0.508076            0.000000   
3               -1.366454             -2.018076            0.166667   
4               -1.829775             -2.621184            0.000000   
..                    ...                   ...                 ...   
758             -0.979703             -0.514252            8.000000   
759              0.581949              1.955626            0.000000   
760              1.769959              1.728446            0.000000   
761              3.106351              4.530658           10.000000   
762              0.602398             -0.155730            1.000000   

     Avgwrongpct-up  Avgtimeperact-up  AvgPrev3Count-up  AvgPrev5Count-up  \
0          0.000000         12.000000          0.000000          0.000000   
1          0.000000          8.750000          0.000000          0.000000   
2          0.000000         26.083333          0.000000          0.000000   
3          0.166667          4.902778          0.000000          0.000000   
4          0.000000          5.875000          0.000000          0.000000   
..              ...               ...               ...               ...   
758        4.000000         17.500000          2.000000          4.000000   
759        0.000000          6.533333          0.000000          0.000000   
760        0.000000         17.076923          0.000000          0.000000   
761        0.000000         16.777778          0.000000          0.000000   
762        0.333333          8.148148          0.333333          0.333333   

     Avg recent5wrong  Avgmanywrong-up  
0            1.000000              0.0  
1            0.000000              0.0  
2            1.000000              0.0  
3            0.000000              0.0  
4            0.000000              0.0  
..                ...              ...  
758          0.000000              0.0  
759          0.000000              0.0  
760          2.000000              0.0  
761          0.000000              0.0  
762          1.666667              0.0  

[763 rows x 14 columns]

In [97]:
# creat a dictionary

group_dict = {}
groups = np.array([])

for index, row in X.iterrows():
    student_id = row['Unique-id']
    if student_id not in group_dict:
        group_dict[student_id] = index
    groups = np.append(groups, group_dict[student_id])

In [98]:
# create the batching variable 
# creat 10 folds

gkf = GroupKFold(n_splits=10)

In [99]:
# kappa and accuracy value based on the Decision Tree

kappa_values = list()

for train_index, test_index in gkf.split(X3, K, groups=groups):
    X_train = X3.iloc[train_index]
    X_test  = X3.iloc[test_index]
    y_train = K[train_index]
    y_test  = K[test_index]
    clf = tree.DecisionTreeClassifier(min_samples_split=10,random_state=5)
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    kappa = cohen_kappa_score(y_test,predictions)
    kappa_values.append(kappa)
     
print(mean(kappa_values))

clf = tree.DecisionTreeClassifier(min_samples_split=10,random_state=5)
cv_f = KFold(n_splits=10, random_state=5, shuffle = True)
scores = cross_val_score(clf, X3, K, cv=cv_f)
print("Accuracy: %.2f%%(%.2f%%)" % (scores.mean()*100, scores.std()*100))

0.13612931721833788
Accuracy: 95.54%(1.88%)


In [100]:
# kappa and accuracy value based on the Naive Bayes


kappa_values = list()

for train_index, test_index in gkf.split(X3, K, groups=groups):
    X_train = X3.iloc[train_index]
    X_test  = X3.iloc[test_index]
    y_train = K[train_index]
    y_test  = K[test_index]
    gnb = GaussianNB()
    gnb.fit(X_train, y_train.ravel())
    predictions = gnb.predict(X_test)
    kappa = cohen_kappa_score(y_test,predictions)
    kappa_values.append(kappa)
    
print(mean(kappa_values))


gnb = GaussianNB()
cv_f = KFold(n_splits=10, random_state=5, shuffle = True)
scores = cross_val_score(gnb, X3, K, cv=cv_f)
print("Accuracy: %.2f%%(%.2f%%)" % (scores.mean()*100, scores.std()*100))

0.09557872366385782
Accuracy: 78.98%(23.54%)


In [101]:
# kappa and accuracy value based on the XGBoosting


kappa_values = list()

for train_index, test_index in gkf.split(X3, K, groups=groups):
    X_train = X3.iloc[train_index]
    X_test  = X3.iloc[test_index]
    y_train = K[train_index]
    y_test  = K[test_index]
    xgb = XGBClassifier(learning_rate=0.5, n_estimators=200, random_state=5)
    xgb.fit(X_train, y_train)
    predictions = xgb.predict(X_test)
    kappa = cohen_kappa_score(y_test,predictions)
    kappa_values.append(kappa)

print(mean(kappa_values))

xgb = XGBClassifier(learning_rate=0.5, n_estimators=200, random_state=5)
cv_f = KFold(n_splits=10, random_state=5, shuffle = True)
scores = cross_val_score(xgb, X3, K, cv=cv_f)
print("Accuracy: %.2f%%(%.2f%%)" % (scores.mean()*100, scores.std()*100))

0.19587207352915098
Accuracy: 96.46%(1.57%)
